In [ ]:
# 게임을 실행하고 실행 결과값을 {"플레이어" : [선택], [gain],[loss],[gain+loss]} 형태의 Json 파일로 저장

import numpy as np
import json

class IGTInteractive:
    def __init__(self, n_trials=100):
        self.name = input("참가자 이름을 입력하세요: ")#참가자 이름 입력
        self.n_trials = n_trials
        self.money = 2000
        self.data = {
            self.name: [[], [], [], []]  # choices, gains, losses, rewards
        }

        self.positive_rewards = {
            0: 100,  # A
            1: 100,  # B
            2:  50,  # C
            3:  50   # D
        }

        self.negative_rewards = {
            0: lambda: np.random.choice([0, 150, 200, 250, 300, 350], p=[1/6]*6),
            1: lambda: np.random.choice([0, 1250], p=[0.9, 0.1]),
            2: lambda: np.random.choice([0, 25, 75, 50], p=[0.5, 0.1, 0.1, 0.3]),
            3: lambda: np.random.choice([0, 250], p=[0.9, 0.1])
        }

    def run(self):
        print(f"\n이 과제의 목표는 최대한 많은 돈을 얻는 것입니다.")
        print(f"각 카드에는 보상과 손실이 있습니다.")
        print(f"4개의 카드 덱(A, B, C, D)은 각각 돈을 얻기에 유리하거나 불리한 카드들로 구성되어 있습니다.")
        print(f"매 시행마다 키보드를 통해 4개의 카드 덱들 중에서 카드 한 개를 뽑아야 합니다.")
        print(f"키보드에서 덱 A: 1, 덱 B: 2, 덱 C: 3, 덱 D: 4를 선택하세요.")
        print(f"\n게임을 시작하겠습니다. (현재 금액: 2000$)\n")

        for t in range(1, self.n_trials + 1):
            while True:
                try:
                    action = int(input(f"{t}번째 선택 : 덱 선택 (1: A, 2: B, 3: C, 4: D): ")) - 1
                    if action in [0, 1, 2, 3]:
                        break
                    else:
                        print("1~4 사이의 숫자를 입력하세요.")
                except ValueError:
                    print("숫자를 입력하세요.")

            gain = self.positive_rewards[action]
            loss = self.negative_rewards[action]()
            reward = gain - loss
            self.money += reward

            self.data[self.name][0].append(action + 1)
            self.data[self.name][1].append(gain)
            self.data[self.name][2].append(-loss)
            self.data[self.name][3].append(reward)

            print(f"   선택한 덱: {chr(65 + action)} | 보상: {gain}$ | 손실: {-1 * loss}$ | 현재 금액: {self.money}$\n")

        print("게임이 종료되었습니다.")
        result = self.data
        save_result_to_json(result, filename = f"{self.name}_result.json")#유저 이름으로 파일 이름 저장
        return 

def convert_numpy_types(obj):
    if isinstance(obj, dict):
        return {k: convert_numpy_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(v) for v in obj]
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    else:
        return obj

def save_result_to_json(data, filename="igt_result.json"):
    data_clean = convert_numpy_types(data)
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data_clean, f, ensure_ascii=False, indent=4)

igt = IGTInteractive(n_trials=100)
igt.run()